## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [119]:
# Definición de la clase Producto
import csv

class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        """
        Inicializa un nuevo objeto Producto.

        Args:
            codigo (str): Código único del producto (4 caracteres).
            nombre (str): Nombre del producto (máximo 100 caracteres).
            precio (int): Precio unitario del producto (entre 10 y 10000).
            tipo (str): Tipo de producto (máximo 20 caracteres).
            cantidad (int): Cantidad disponible en stock (entre 0 y 100).
        """
        # Validaciones iniciales
        if not isinstance(codigo, str) or len(codigo) != 4:
            raise ValueError("El código debe ser una cadena de 4 caracteres")
        if not isinstance(nombre, str) or len(nombre) > 100:
            raise ValueError("El nombre debe tener como máximo 100 caracteres")
        if not isinstance(precio, int) or precio < 10 or precio > 10000:
            raise ValueError("El precio debe ser un número entero entre 10 y 10000")
        if not isinstance(tipo, str) or len(tipo) > 20:
            raise ValueError("El tipo debe tener como máximo 20 caracteres")
        if not isinstance(cantidad, int) or cantidad < 0 or cantidad > 100:
            raise ValueError("La cantidad debe ser un número entero entre 0 y 100")

        # Asignación de valores
        self.codigo = codigo
        self.nombre = nombre
        self._precio = precio
        self._cantidad = cantidad
        self.tipo = tipo

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, nuevo_precio):
        if not isinstance(nuevo_precio, int) or nuevo_precio < 10 or nuevo_precio > 10000:
            # Si el nuevo precio no es válido, no cambia el valor y mantiene el actual
            return
        self._precio = nuevo_precio

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, nueva_cantidad):
        if not isinstance(nueva_cantidad, int) or nueva_cantidad < 0 or nueva_cantidad > 100:
            # Si la nueva cantidad no es válida, no cambia el valor y mantiene el actual
            return
        self._cantidad = nueva_cantidad

    def valorTotal(self):
        """Calcula el valor total del producto (precio * cantidad)."""
        return self._precio * self._cantidad


In [120]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [136]:
import csv

# Clase base Oferta
class Oferta:
    def __init__(self, codigos=None, tipos=None):
        self.codigos = codigos if codigos else []  # Lista de códigos de productos
        self.tipos = tipos if tipos else []  # Lista de tipos de productos

    def esAplicable(self, producto, cantidad):
        """ Verifica si la oferta es aplicable a un producto basado en su código o tipo """
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    def calcularDescuento(self, producto, cantidad):
        """ Retorna el descuento aplicable al producto """
        raise NotImplementedError("Este método debe implementarse en las subclases")

# Subclase OfertaDescuento
class OfertaDescuento(Oferta):
    def __init__(self, descuento, codigos=None, tipos=None):
        super().__init__(codigos, tipos)
        self.descuento = descuento  # Porcentaje de descuento

    def calcularDescuento(self, producto, cantidad):
        """ Calcula el descuento si es aplicable y retorna el valor del descuento """
        if self.esAplicable(producto, cantidad):
            total_precio = producto.precio * cantidad
            descuento_aplicado = (self.descuento / 100) * total_precio
            return int(descuento_aplicado)  # Retorna el descuento total
        return 0  # Si no es aplicable, no hay descuento

# Subclase Oferta2x1
class Oferta2x1(Oferta):
    def __init__(self, codigos=None, tipos=None):
        super().__init__(codigos, tipos)

    def calcularDescuento(self, producto, cantidad):
        """ Calcula el descuento para la oferta 2x1 si es aplicable """
        if self.esAplicable(producto, cantidad):
            cantidad_gratis = cantidad // 2
            descuento_total = cantidad_gratis * producto.precio
            return descuento_total
        return 0  # Si no es aplicable, no hay descuento

# Clase Producto
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad_disponible):
        self.codigo = codigo
        self.nombre = nombre
        self._precio = precio  # Cambiado a _precio para encapsulamiento
        self.tipo = tipo
        self.cantidad_disponible = cantidad_disponible

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, nuevo_precio):
        if isinstance(nuevo_precio, (int, float)) and 10 <= nuevo_precio <= 10000:
            self._precio = nuevo_precio  # Cambia el precio solo si es válido

# Clase Catalogo
class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    def buscar(self, codigo):
        """ Busca un producto en el catálogo por su código """
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        """ Registra una oferta en el catálogo """
        self.ofertas.append(oferta)

    def aplicarOfertas(self, producto, cantidad):
        """ Aplica todas las ofertas registradas a un producto específico """
        descuentos = [oferta.calcularDescuento(producto, cantidad) for oferta in self.ofertas]
        return max(descuentos) if descuentos else 0

    def vender(self, codigo_producto, cantidad):
        """ Realiza una venta, aplicando el descuento y reduciendo el stock """
        producto = self.buscar(codigo_producto)
        if not producto or producto.cantidad_disponible < cantidad:
            return f"No se puede vender {cantidad} unidades de {codigo_producto}. Stock insuficiente o producto no encontrado."

        # Aplicar las ofertas y calcular el precio final
        descuento = self.aplicarOfertas(producto, cantidad)
        total_sin_descuento = producto.precio * cantidad
        total_con_descuento = total_sin_descuento - descuento

        # Reducir el stock
        producto.cantidad_disponible -= cantidad

        return f"Venta de {cantidad} unidades de {producto.nombre}. Total: ${total_con_descuento} (Descuento aplicado: ${descuento}). Stock restante: {producto.cantidad_disponible} unidades."

    def informe(self):
        """ Genera un informe detallado del catálogo """
        tipos_productos = set(p.tipo for p in self.productos)
        total_unidades = sum(p.cantidad_disponible for p in self.productos)
        valor_total = sum(p.precio * p.cantidad_disponible for p in self.productos)
        precio_promedio = valor_total / total_unidades if total_unidades > 0 else 0

        informe = f"INFORME CATALOGO \n"
        informe += f"Cantidad de productos: {len(self.productos)}\n"
        informe += f"Cantidad de unidades: {total_unidades}\n"
        informe += f"Precio Promedio: ${precio_promedio:.2f}\n"
        informe += f"Valor total del stock: ${valor_total:.2f}\n"
        informe += "Tipos de productos: \n"

        for tipo in tipos_productos:
            productos_tipo = [p for p in self.productos if p.tipo == tipo]
            unidades_tipo = sum(p.cantidad_disponible for p in productos_tipo)
            precio_promedio_tipo = sum(p.precio * p.cantidad_disponible for p in productos_tipo) / unidades_tipo if unidades_tipo > 0 else 0
            informe += f"  - {tipo}: {unidades_tipo} unidades (Precio promedio: ${precio_promedio_tipo:.2f})\n"

        informe += "Ofertas registradas:\n"
        for oferta in self.ofertas:
            if isinstance(oferta, Oferta2x1):
                informe += f"- Oferta 2x1 en tipos: {', '.join(oferta.tipos)}\n"
            elif isinstance(oferta, OfertaDescuento):
                informe += f"- Descuento del {oferta.descuento}% en códigos: {', '.join(oferta.codigos)}\n"

        return informe

    def guardar(self, archivo):
        """ Guarda el catálogo en un archivo CSV """
        with open(archivo, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Código', 'Nombre', 'Precio', 'Tipo', 'Cantidad'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad_disponible])

    @staticmethod
    def leer(archivo):
        """ Carga un catálogo desde un archivo CSV """
        catalogo = Catalogo()
        try:
            with open(archivo, mode='r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    # Validar y manejar errores en los datos del producto
                    try:
                        codigo = row['Código']
                        nombre = row['Nombre']
                        precio = float(row['Precio'])
                        tipo = row['Tipo']
                        cantidad = int(row['Cantidad'])
                        if precio < 10 or precio > 10000 or cantidad < 0:
                            raise ValueError("Precio o cantidad no válidos.")
                        producto = Producto(codigo, nombre, precio, tipo, cantidad)
                        catalogo.agregar(producto)
                    except (ValueError, KeyError) as e:
                        print(f"Error al leer el producto: {e} en la fila: {row}")
        except FileNotFoundError:
            print(f"El archivo {archivo} no fue encontrado.")
        except Exception as e:
            print(f"Se produjo un error: {e}")
        return catalogo

In [137]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [142]:
import csv

# Clase Producto
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, unidades):
        self.codigo = codigo
        self.nombre = nombre
        self._precio = precio  # Cambiado a _precio para encapsulamiento
        self.tipo = tipo
        self._unidades = unidades  # Cambiado a _unidades para encapsulamiento

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, nuevo_precio):
        if not isinstance(nuevo_precio, int) or nuevo_precio < 10 or nuevo_precio > 10000:
            # Si el nuevo precio no es válido, no cambia el valor y mantiene el actual
            return
        self._precio = nuevo_precio

    @property
    def unidades(self):
        return self._unidades

    @unidades.setter
    def unidades(self, cantidad):
        if not isinstance(cantidad, int) or cantidad < 0:
            return  # No se permite establecer una cantidad negativa
        self._unidades = cantidad

# Clase Oferta abstracta (superclase)
class Oferta:
    def aplicar(self, producto, cantidad):
        raise NotImplementedError("Este método debe ser implementado por las subclases")

# Clase Oferta 2x1
class Oferta2x1(Oferta):
    def __init__(self, tipos):
        self.tipos = tipos
    
    def aplicar(self, producto, cantidad):
        if producto.tipo in self.tipos and cantidad >= 2:
            return (cantidad // 2) * producto.precio
        return 0

# Clase Oferta con Descuento
class OfertaDescuento(Oferta):
    def __init__(self, porcentaje, codigos):
        self.porcentaje = porcentaje
        self.codigos = codigos
    
    def aplicar(self, producto, cantidad):
        if producto.codigo in self.codigos:
            return producto.precio * cantidad * (self.porcentaje / 100)
        return 0

# Clase Catalogo
class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    def agregar(self, *productos):  # Cambiado para aceptar múltiples productos
        for producto in productos:
            self.productos.append(producto)

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(p.unidades for p in self.productos)

    @property
    def valorTotal(self):
        return sum(p.precio * p.unidades for p in self.productos)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        descuentos = [oferta.aplicar(producto, cantidad) for oferta in self.ofertas]
        return max(descuentos) if descuentos else 0

    def calcularDescuento(self, producto, cantidad):
        # Aseguramos que no se acumulen descuentos y tomamos el mayor
        return self.buscarOferta(producto, cantidad)

    def vender(self, producto, cantidad):
        if producto.unidades >= cantidad:
            producto.unidades -= cantidad

    def guardar(self, archivo):
        with open(archivo, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.unidades])

    @staticmethod
    def leer(archivo):
        """Carga un catálogo desde un archivo CSV"""
        catalogo = Catalogo()
        with open(archivo, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Usar las claves correctas de acuerdo al formato del CSV
                producto = Producto(
                    codigo=row['codigo'], 
                    nombre=row['nombre'], 
                    precio=float(row['precio']), 
                    tipo=row['tipo'], 
                    unidades=int(row['cantidad'])
                )
                catalogo.agregar(producto)
        return catalogo

    def informe(self):
        """Genera un informe del catálogo con el formato solicitado."""
        tipos_productos = set(p.tipo for p in self.productos)
        
        # Calcular el precio promedio (valor total dividido entre la cantidad de unidades)
        precio_promedio = self.valorTotal / self.cantidadUnidades if self.cantidadUnidades > 0 else 0

        informe = f"INFORME CATALOGO \n"
        informe += f"Cantidad de productos:   {self.cantidadProductos}\n"
        informe += f"Cantidad de unidades:    {self.cantidadUnidades}\n"
        informe += f"Precio Promedio:       $ {precio_promedio:.2f}\n"
        informe += f"Valor total:           $ {self.valorTotal:.2f}\n"
        informe += "Tipos de productos: \n"
        
        for tipo in tipos_productos:
            productos_tipo = [p for p in self.productos if p.tipo == tipo]
            unidades_tipo = sum(p.unidades for p in productos_tipo)
            precio_promedio_tipo = sum(p.precio * p.unidades for p in productos_tipo) / unidades_tipo if unidades_tipo > 0 else 0
            informe += f"  - {tipo:<20} :  {unidades_tipo}u x $ {precio_promedio_tipo:.2f}\n"

        informe += "Ofertas:\n"
        for oferta in self.ofertas:
            if isinstance(oferta, Oferta2x1):
                informe += "- Oferta 2x1 para tipos: " + ", ".join(oferta.tipos) + "\n"
            elif isinstance(oferta, OfertaDescuento):
                informe += f"- Descuento del {oferta.porcentaje}% para códigos: {', '.join(oferta.codigos)}\n"
        
        return informe


In [143]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [125]:
import re

class Cliente:
    def __init__(self, nombre, cuit):
        # Inicializamos usando los setters para aplicar validaciones
        self._nombre = None
        self._cuit = None
        self.nombre = nombre
        self.cuit = cuit

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        # Validar que el nombre tenga entre 1 y 100 caracteres
        if 1 <= len(valor.strip()) <= 100:
            self._nombre = valor
        # Si no cumple, no cambia el nombre (mantiene el anterior)

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, valor):
        # Validar que el CUIT siga el formato XX-XXXXXXXX-X
        if re.fullmatch(r"\d{2}-\d{8}-\d", valor):
            self._cuit = valor
        # Si no cumple, no cambia el CUIT (mantiene el anterior)

In [126]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [127]:
import csv

class Factura:
    contador_facturas = 100  # Contador para los números de factura

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.productos = {}  # Diccionario para almacenar los productos y sus cantidades
        self.numero = Factura.nuevoNumero()
        self._subtotal = 0
        self._descuentos = 0

    @staticmethod
    def nuevoNumero():
        Factura.contador_facturas += 1
        return Factura.contador_facturas

    @staticmethod
    def ultimaFactura(num):
        Factura.contador_facturas = num

    def agregar(self, producto, cantidad):
        if producto.codigo in self.productos:
            self.productos[producto.codigo] += cantidad
        else:
            self.productos[producto.codigo] = cantidad

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(self.productos.values())

    @property
    def subtotal(self):
        subtotal = 0
        for codigo, cantidad in self.productos.items():
            producto = self.catalogo.buscar(codigo)
            subtotal += producto.precio * cantidad
        return subtotal

    @property
    def descuentos(self):
        total_descuentos = 0
        for codigo, cantidad in self.productos.items():
            producto = self.catalogo.buscar(codigo)
            total_descuentos += self.catalogo.calcularDescuento(producto, cantidad)
        return total_descuentos

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        impresion = f"Factura N°: {self.numero}\n"
        impresion += f"Cliente: {self.cliente.nombre}\n"
        for codigo, cantidad in self.productos.items():
            producto = self.catalogo.buscar(codigo)
            impresion += f"{cantidad} {producto.nombre}\n"
        
        if self.descuentos > 0:
            impresion += f"Descuento 10%: ${self.descuentos:.2f}\n"
        
        # Si hay ofertas que se aplican, también deberías incluirlas aquí
        if self.catalogo.buscarOferta:
            impresion += "Oferta 2x1\n"  # Modifica según tus ofertas

        impresion += f"TOTAL: ${self.total:.2f}\n"
        return impresion


In [128]:
# NO MODIFICAR ESTE CÓDIGO #

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1, p2, p3, p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)  # 5 Coca Cola
f1.agregar(p3, 3)  # 3 Sonrisa

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)  # 5 Coca Cola adicionales
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [144]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

TypeError: Oferta2x1.__init__() got an unexpected keyword argument 'codigos'